In [2]:
import numpy as np
import pandas as pd
import os
import warnings
import bellatrex as btrex
print(btrex.__version__)
# reduce MAX_TEST_SAMPLES for quick code testing
MAX_TEST_SAMPLES = 2

PLOT_GUI = False
'''  levels of verbosity in this script:
    - >= 0.0: sample i and fold j print, ensemble and local score.
    - >= 1.0: print best params, their achieved fidelity,
              and the scoring method used to compute such performance
    - >= 2.0 print final tree idx cluster sizes
    - >= 3.0: plot representation of the extracted trees (two plots)
    - >= 4.0 plot trees with GUI (if PLOT_GUI == True)
    - >= 4.0 plot trees without GUI (if PLOT_GUI == False)
    - >= 5.0: print params and performance during GridSearch
'''

##########################################################################
root_folder = os.getcwd()
print(root_folder)

0.1.4
c:\Users\u0135479\Documents\GitHub\Bellatrex_pip


In [3]:
from bellatrex.datasets import load_mtr_data, load_mlc_data
from bellatrex.datasets import load_survival_data, load_binary_data, load_regression_data

X, y = load_binary_data(return_X_y=True)
X, y = load_regression_data(return_X_y=True)
X, y = load_survival_data(return_X_y=True)
X, y = load_mlc_data(return_X_y=True)
# X, y = load_mtr_data(return_X_y=True)
# X, y = X[::2], y[::2]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

from bellatrex.utilities import get_auto_setup
SETUP = get_auto_setup(y)
print('Detected prediction task \'SETUP\':', SETUP)

Detected prediction task 'SETUP': multi-label


In [6]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

### instantiate original R(S)F estimator, works best with some pruning. 
if SETUP.lower() in 'survival':
    clf = RandomSurvivalForest(n_estimators=100, min_samples_split=10,
                                n_jobs=-2, random_state=0)

elif SETUP.lower() in ['binary', 'multi-label']:
    clf = RandomForestClassifier(n_estimators=100, min_samples_split=5,
                                n_jobs=-2, random_state=0)
    
elif SETUP.lower() in ['regression', 'multi-target']:
    clf = RandomForestRegressor(n_estimators=100, min_samples_split=5,
                                n_jobs=-2, random_state=0)


from bellatrex import BellatrexExplain
from bellatrex.wrapper_class import EnsembleWrapper, tree_list_to_model, DT_to_dict

clf.fit(X_train, y_train)


['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__sklearn_clone__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_build_request_for_signature', '_check_feature_names', '_check_n_features', '_compute_oob_predictions', '_estimator_type', '_get_default_requests', '_get_metadata_request', '_get_oob_predictions', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_parameter_constraints', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score_and_attributes', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_params', '_validate_y_class_wei

In [10]:
# dictionary testing for BELLATREX here
tree_list = []
    
for t in range(clf.n_estimators):
        
    tree_dict = DT_to_dict(clf, t, 'auto',
                                T_to_bin=None)
    tree_list.append(tree_dict)
        
clf1 = tree_list_to_model(tree_list) #normally, you load the dict in this format
# adapt the dict into smth compatible with Bellatrex
clf2 = EnsembleWrapper(clf1)


In [11]:
#fit RF here. The hyperparameters are given
# compatible with trained model clf, and with a wrapped dictionary as in clf1        
Btrex_fitted = BellatrexExplain(clf, set_up='auto',
                                verbose=3).fit(X_train, y_train)

import matplotlib.pyplot as plt
from bellatrex.visualisation import read_rules, plot_rules

for i in range(MAX_TEST_SAMPLES): #iterate for the first few samples in the test set

    print(f"Explaining sample i={i}")
    
    # Store the extracted rules in dedicated folder, if needed:
    explan_dir = os.path.join(root_folder, "explanations-out")
    os.makedirs(explan_dir, exist_ok=True)
    FILE_OUT = os.path.join(explan_dir, "Rules_"+str(SETUP)+'_id'+str(i)+'.txt')

    # call the .explain method. The hyperparameters are given from the .fit step
    # sample_info, fig, axis = Btrex_fitted.explain(X_test, i, show=True, plot_gui=False,
                                                #   out_file=FILE_OUT)

    tuned_method = Btrex_fitted.explain(X_test, i)
    tuned_method.plot_overview(show=True)


Model is already fitted, building explanation.
oracle_sample is: None
Automatically setting prediction task to:  multi-label
Explaining sample i=0
<class 'bellatrex.TreeExtraction_class.TreeExtraction'> dir: ['BINARY_KEYS', 'MSA_KEYS', 'MTC_KEYS', 'MTR_KEYS', 'RAND_SEED', 'REGRESSION_KEYS', 'SURVIVAL_KEYS', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'calcul_tree_proximity_loss', 'clf', 'cluster_sizes', 'dim_reduction', 'dissim_method', 'extract_final_trees', 'feature_represent', 'fidelity_measure', 'final_trees_idx', 'get_params', 'jaccard_pair_distance', 'local_prediction', 'main_fit', 'n_clusters', 'n_dims', 'n_trees', 'oracle_prediction', 'oracle_sample', 'output_explain', 'pre_select_lo

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
if clf.n_outputs_ == 1:
    FILE_OUT_EXTRA = f"{os.path.splitext(FILE_OUT)[0]}-extra{os.path.splitext(FILE_OUT)[1]}"

    rules, preds, baselines, weights, other_preds = read_rules(file=FILE_OUT,
                                                            file_extra=FILE_OUT_EXTRA)
    preds_distr = y_train # if SETUP == 'binary' else None

    fig, axs = plot_rules(rules, preds, baselines, weights, 
                max_rulelen=6, other_preds=other_preds, preds_distr=preds_distr,
    )
    plt.show()
    print('#'*30)
y_pred = clf.predict_proba(X_test.iloc[[i]])[:,1] if 'predict_proba' in dir(clf) and clf.n_outputs_ == 1 else clf.predict(X_test.iloc[[i]])
if y_pred.ndim == 1:
    print(f'Bellatrex surrogate prediction: {float(sample_info.prediction):.3g}')
    print(f'Original, black-box prediction: {float(y_pred):.3g}')